In [3]:
import keras
from keras.models import Model, Sequential
import numpy as np
from keras.optimizers import SGD, Adam
from keras.layers import Dense, LSTM, Input, Reshape, merge
import numpy as np
from keras.layers.wrappers import TimeDistributed
from keras.layers.core import TimeDistributedDense
import sys
import keras.backend as K
import tensorflow as tf

In [4]:
global _SESSION
configtensor = tf.ConfigProto(allow_soft_placement=True)
configtensor.gpu_options.allow_growth = True
_SESSION = tf.Session(config=configtensor)
K.set_session(_SESSION)

# 1. Loading and testing input data

In [5]:
print ('Loading training data')
inputFile  = "/home/madli/Downloads/1. keras audio gan - audio-GAN-master/datasets/YourMusicLibraryNP"
X_train = np.load(inputFile + '_x.npy')
Y_train = np.load(inputFile + '_y.npy')


Loading training data


In [6]:
X_train = X_train[:100]
Y_train = Y_train[:100]

In [7]:
input_shape = X_train.shape
noise_train = np.random.random(input_shape) 
num_hidden_dimensions = 2
sz=(X_train.shape[1], X_train.shape[2])
n_epochs = 10 #What does this number do?
batch_size = 20 #How many samples of generated and real audio are fed to the discriminator

# 2. Defining and testing the generator

In [8]:
def get_generative_model(num_recurrent_units=1):
    model = Sequential()
    #This layer converts frequency space to hidden space
    model.add( TimeDistributed( Dense( num_hidden_dimensions ), input_shape=sz ) )
    for cur_unit in xrange(num_recurrent_units):
        model.add(LSTM(input_dim=num_hidden_dimensions, output_dim=num_hidden_dimensions, return_sequences=True))
    model.add(Dense(input_dim=num_hidden_dimensions, output_dim=num_hidden_dimensions)) #W_regularizer = l2(sys.argv[1])
    #This layer converts hidden space back to frequency space
    model.add(TimeDistributedDense(input_dim=num_hidden_dimensions, output_dim=X_train.shape[2]))
    return model

In [11]:
def get_generative_model():
    first_input = Input(shape=sz)
    first_dense = Dense(output_dim=num_hidden_dimensions)(first_input)
    
    second_input = Input(shape = sz)
    second_dense = Dense(output_dim=num_hidden_dimensions)(second_input)
    
    merge_one = merge([first_dense, second_dense], mode="concat", concat_axis=2)
    merge_one = Dense(output_dim=num_hidden_dimensions)(merge_one)
    
    lstm_out = LSTM(input_dim=num_hidden_dimensions, output_dim=num_hidden_dimensions, return_sequences=True)(first_dense)
    third_dense = Dense(num_hidden_dimensions)(lstm_out)
    outputs = TimeDistributedDense(X_train.shape[2])(second_dense)
    model = Model(input=[first_input,second_input], output = outputs)
    # Why four values after merging? Why can't change the output dimensions?
    return model

In [9]:
generator = get_generative_model()

/home/madli/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:1206: UserWarning: `TimeDistributedDense` is deprecated, And will be removed on May 1st, 2017. Please use a `Dense` layer instead.
  warnings.warn('`TimeDistributedDense` is deprecated, '


In [10]:
generator.compile(loss='binary_crossentropy', optimizer='sgd')
predictions2 = generator.predict (X_train)
predictions2.shape
#labelsdiscfalse = np.ones(predictionsdisc.shape, dtype=np.int) #predictionsdisc.shape asendada

(100, 8, 4096)

In [11]:
generator.compile(loss='binary_crossentropy', optimizer='sgd')
generator.fit(X_train, Y_train)
predictions = generator.predict(X_train)
predictions.shape

Epoch 1/10
100/100 [==============================] - 0s - loss: 0.0108     
Epoch 2/10
100/100 [==============================] - 0s - loss: -0.0085     
Epoch 3/10
100/100 [==============================] - 0s - loss: -0.0248     
Epoch 4/10
100/100 [==============================] - 0s - loss: -0.0310     

(100, 8, 4096)

# 3. Defining and testing the discriminator

In [13]:
# takes in the concatenated input and outputs a value
def get_disc_model():    
    first_input = Input(shape=(sz[0],sz[1]*2))
    first_dense = Dense(output_dim=3)(first_input)

    second_input = Input(shape=sz)
    second_dense = Dense(output_dim=3)(second_input)

    merge_one = merge([first_dense, second_dense], mode="concat", concat_axis=2)
    merge_one = Dense(1, activation='sigmoid')(merge_one)
    model = Model(input=[first_input,second_input], output = merge_one) #[first_input, second_input]
    # Why four values after merging? Why can't change the output dimensions?
    return model

In [16]:
fake_part = generator.predict(inputdata)
inputdata = X_train[0*batch_size:(0+1)*batch_size]
real_part = Y_train[0*batch_size:(0+1)*batch_size]
real_pairs = np.concatenate((inputdata,real_part),axis=2) 
fake_pairs = np.concatenate((inputdata,fake_part),axis=2) 
labels = np.ones((10,8,1), dtype=np.int)

In [18]:
print real_pairs[0]
print "other"
print fake_pairs[0]

[[ 0.03053178  0.10942413  0.09317692 ..., -0.04423774 -0.22764404
   0.003853  ]
 [-0.02000679  0.01765924 -0.25233278 ...,  0.09669302  0.32945899
   0.06675315]
 [-0.10424472 -0.12149944 -0.2824247  ..., -0.32801883  0.35108975
   0.04574882]
 ..., 
 [ 0.47509923  0.37239508  0.1958198  ...,  0.03008591 -0.05190928
   0.01690779]
 [-0.12812997 -0.07219134  0.72107097 ..., -0.25292542  0.13796815
   0.21532358]
 [-0.23903382 -0.03900657  0.99542972 ...,  0.15476935 -0.07502141
  -0.06414316]]
other
[[ 0.03053178  0.10942413  0.09317692 ...,  0.00880897 -0.00749003
   0.00327377]
 [-0.02000679  0.01765924 -0.25233278 ...,  0.00924417 -0.0078593
   0.00291241]
 [-0.10424472 -0.12149944 -0.2824247  ...,  0.00849269 -0.00813041
   0.00255781]
 ..., 
 [ 0.47509923  0.37239508  0.1958198  ...,  0.00819626 -0.00618548
   0.00462747]
 [-0.12812997 -0.07219134  0.72107097 ...,  0.00958179 -0.00650731
   0.00439645]
 [-0.23903382 -0.03900657  0.99542972 ...,  0.0095436  -0.00705654
   0.003801

In [103]:
discriminator = get_disc_model()
discriminator.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
discriminator.fit (real_pairs, labels)
disc_predictions = discriminator.predict(real_pairs)
disc_predictions.shape

Epoch 1/10
10/10 [==============================] - 0s - loss: 0.9090 - acc: 0.5625
Epoch 2/10
10/10 [==============================] - 0s - loss: 0.5898 - acc: 0.6500
Epoch 3/10
10/10 [==============================] - 0s - loss: 0.5352 - acc: 0.7750
Epoch 4/10
10/10 [==============================] - 0s - loss: 0.5044 - acc: 0.8375
Epoch 5/10
10/10 [==============================] - 0s - loss: 0.4816 - acc: 0.8500
Epoch 6/10
10/10 [==============================] - 0s - loss: 0.4636 - acc: 0.8625
Epoch 7/10
10/10 [==============================] - 0s - loss: 0.4485 - acc: 0.8875
Epoch 8/10
10/10 [==============================] - 0s - loss: 0.4354 - acc: 0.9250
Epoch 9/10
10/10 [==============================] - 0s - loss: 0.4239 - acc: 0.9375
Epoch 10/10
10/10 [==============================] - 0s - loss: 0.4135 - acc: 0.9500


(10, 8, 1)

In [89]:
discriminator = get_disc_model()

# 4. Putting discriminator together with the generator

Generator takes in a vector, transforms it into a vector with the same size. 
The output of the generator is concatenated with new vector of same size - resulting in a vector that is 2 as big.
Discriminator takes an input that is 2 as big - concatenated parts
Discriminator outputs 1 value

In [166]:
def generator_containing_discriminator(generator, discriminator):
    inputs = Input((8,4096))
    x_generator = generator(inputs) #produces output that is shaped (100, 8, 4096)
    
    #takes the output produced by the generator and output and adds real input to the beginning
    merged = merge([inputs, x_generator], mode='concat',concat_axis=2) 
    discriminator.trainable = False
    x_discriminator = discriminator(merged)
    
    model = Model(input=inputs, output=[x_generator,x_discriminator])
    return model

In [167]:
generator_containing_disciminator = generator_containing_discriminator(generator, discriminator)

In [169]:
results = generator_containing_disciminator.predict(X_train)
results

[array([[[  7.71134626e-04,  -3.42320709e-05,  -8.03471310e-04, ...,
            1.31508487e-03,   9.38146608e-04,   2.14764528e-04],
         [ -2.01083021e-05,  -9.36924829e-04,  -9.88789019e-04, ...,
           -1.47598970e-04,   1.88894453e-04,   5.01716626e-04],
         [ -5.05514955e-03,   2.01874902e-03,   3.43254954e-03, ...,
           -3.15271318e-03,  -4.60994430e-04,   7.57154450e-03],
         ..., 
         [  3.73532041e-03,   8.03576689e-03,   4.12738789e-03, ...,
           -2.80135125e-03,   6.67523942e-04,  -3.49254091e-03],
         [  2.91457470e-03,   7.81282876e-03,   5.51034790e-03, ...,
           -5.03126998e-03,  -1.75578543e-03,  -5.76267019e-03],
         [  2.83952174e-03,   5.53051382e-03,   4.49369894e-03, ...,
           -4.44472162e-03,  -1.99654605e-03,  -5.76119684e-03]],
 
        [[ -1.84520846e-04,  -7.81183480e-05,   6.02692307e-04, ...,
            8.97012942e-05,  -3.72854120e-04,  -2.18529531e-05],
         [ -2.32931372e-04,   3.60343023e-04

In [29]:
'''
def get_generator_containing_disciminator():
    first_input = Input(shape=sz)
    first_dense = Dense(output_dim=num_hidden_dimensions)(first_input)
    lstm_out = LSTM(input_dim=num_hidden_dimensions, output_dim=num_hidden_dimensions, return_sequences=True)(first_dense)
    second_dense = Dense(num_hidden_dimensions)(lstm_out)
    outputs = TimeDistributedDense(X_train.shape[2])(second_dense)

    second_input = outputs
    first_dense = Dense(output_dim=3)(second_input)

    third_input = Input(shape=sz)
    second_dense = Dense(output_dim=3)(third_input)

    merge_one = merge([first_dense, second_dense]) #mode="concat", concat_axis=1
    merge_one = Dense(1, activation='sigmoid')(merge_one)
    model = Model(input=[first_input, third_input], output = merge_one)
    return model

In [34]:
#genanddisc = generator_containing_discriminator(generator, discriminator)

In [11]:
'''
def get_generator_containing_disciminator(generator, discriminator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    return model

In [19]:
#generator_containing_disciminator = get_generator_containing_disciminator(generator, discriminator)

In [43]:
'''
first_input = Input(shape=sz)
first_layer = generator(first_input)
second_input = first_layer

third_input= Input(shape=sz)
second_layer = discriminator(first_layer)
model = Model(input=first_input, output = second_layer)

gan_input = Input(shape=sz)
H = generator(gan_input)
gan_V = discriminator(H)
GAN = Model(gan_input, gan_V)
#GAN.compile(loss='categorical_crossentropy', optimizer=opt)
#GAN.summary()

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-43-3c9b11ca62c1>, line 15)

# 5. Setting optimization parameters 

In [174]:
d_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
g_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
generator.compile(loss='binary_crossentropy', optimizer='sgd')
generator_containing_disciminator.compile(loss='binary_crossentropy', optimizer=g_optim)
discriminator.trainable = True
discriminator.compile(loss='binary_crossentropy', optimizer=d_optim)

# 6. Get total number of batches

In [21]:
n_batches = int(len(X_train)/batch_size)

10

# 7. Get losses

In [2]:
i = 0
def get_losses(i):
    print ('Epoch:', i+1)
    d_losses = []
    g_losses = []
    for index in range(n_batches):
        # 9.1 Get generated samples according to the batch size number
        inputdata = X_train[index*batch_size:(index+1)*batch_size]
        noise = np.random.random(inputdata.shape)
        generated_part = generator.predict(inputdata)
        real_part = Y_train[index*batch_size:(index+1)*batch_size]
        real_pairs = np.concatenate((inputdata,real_part),axis=2)  #maybe not necessary since discriminator takes 2 inputs anyway
        fake_pairs = np.concatenate((inputdata,generated_part),axis=2)
        X = np.concatenate((real_pairs,fake_pairs))
        #print X.shape
        real_labels = np.ones((batch_size,X_train.shape[1],1))
        fake_labels = np.zeros((batch_size,X_train.shape[1],1))
        y = np.concatenate((real_labels,fake_labels),axis=0)
        #print y
        d_loss = discriminator.train_on_batch(X, y) # why does this output 2 numbers???
        #print discriminator.metrics_names
        d_losses.append(d_loss)
        #print d_loss
        pred_temp = discriminator.predict(X)
        #print pred_temp
        discriminator.trainable = False
        #print inputdata.shape
        
        g_loss = generator_containing_disciminator.train_on_batch(inputdata, [real_part,real_labels] )
        g_losses.append(g_loss)
        #print generator_containing_disciminator.metrics_names
        #print g_loss
        #print("batch %d g_loss : %f" % (index, g_loss[1]))
        discriminator.trainable = True
        if index % 10 == 0:
                generator.save_weights('generator'+str(index), True)
                discriminator.save_weights('discriminator'+str(index), True)     
    
    
            # Print how many batches have been trained on 
        sys.stdout.write(' + batch: ' + str(index+1) + '/' + str(n_batches) + '\r')
        # Everything from the buffer will be written in the terminal 
        sys.stdout.flush()
    i += 1
    return d_losses, g_losses

In [3]:
for i in range(n_epochs):   
    d_losses, g_losses = get_losses(i)
    mean_dloss = round(np.mean(d_losses), 2)
    mean_gloss = round(np.mean(g_losses), 2)
    ## REMOVE \N if it is not necessary 
    print ('\n + d_loss:', mean_dloss)
    print (' + g_loss:', mean_gloss)

NameError: name 'n_epochs' is not defined